In [ ]:
storageAccountKey = ''
spark.conf.set('fs.azure.account.key.myazuredemostorageacc.dfs.core.windows.net',storageAccountKey)

In [ ]:
source_json_filepath = 'https://date.nager.at/api/v3/PublicHolidays/2025/US'
target_csv_filepath = 'abfss://assignment-container@myazuredemostorageacc.dfs.core.windows.net/bronze_assignment/public_holidays/csv'

In [ ]:
import pandas

In [ ]:
source_json_file_pandas_df = pandas.read_json(source_json_filepath)

In [ ]:
from pyspark.sql.functions import *
source_json_file_sparks_df = spark.createDataFrame(source_json_file_pandas_df)

In [ ]:
print(source_json_file_sparks_df)
display(source_json_file_sparks_df)

DataFrame[date: timestamp, localName: string, name: string, countryCode: string, fixed: boolean, global: boolean, counties: array<string>, launchYear: double, types: array<string>]


date,localName,name,countryCode,fixed,global,counties,launchYear,types
2025-01-01T00:00:00Z,New Year's Day,New Year's Day,US,false,true,null,null,List(Public)
2025-01-20T00:00:00Z,"Martin Luther King, Jr. Day","Martin Luther King, Jr. Day",US,false,true,null,null,List(Public)
2025-02-12T00:00:00Z,Lincoln's Birthday,Lincoln's Birthday,US,false,false,"List(US-CA, US-CT, US-IL, US-IN, US-KY, US-MI, US-NY, US-MO, US-OH)",null,List(Observance)
2025-02-17T00:00:00Z,Washington's Birthday,Presidents Day,US,false,true,null,null,List(Public)
2025-04-18T00:00:00Z,Good Friday,Good Friday,US,false,false,"List(US-CT, US-DE, US-HI, US-IN, US-KY, US-LA, US-NC, US-ND, US-NJ, US-TN)",null,List(Public)
2025-04-18T00:00:00Z,Good Friday,Good Friday,US,false,false,List(US-TX),null,List(Optional)
2025-05-08T00:00:00Z,Truman Day,Truman Day,US,false,false,List(US-MO),null,"List(School, Authorities)"
2025-05-26T00:00:00Z,Memorial Day,Memorial Day,US,false,true,null,null,List(Public)
2025-06-19T00:00:00Z,Juneteenth National Independence Day,Juneteenth National Independence Day,US,false,true,null,null,List(Public)
2025-07-04T00:00:00Z,Independence Day,Independence Day,US,false,true,null,null,List(Public)


In [ ]:
from pyspark.sql.functions import col, concat_ws
source_json_file_sparks_df = source_json_file_sparks_df.withColumn(
    "counties",
    concat_ws(",", col("counties"))
)


In [ ]:
source_json_file_sparks_df.selectExpr(*[f"cast({col} as string) as {col}" for col in source_json_file_sparks_df.columns]).write.mode('overwrite').csv(target_csv_filepath)

In [ ]:
source_json_file_sparks_df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- localName: string (nullable = true)
 |-- name: string (nullable = true)
 |-- countryCode: string (nullable = true)
 |-- fixed: boolean (nullable = true)
 |-- global: boolean (nullable = true)
 |-- counties: string (nullable = false)
 |-- launchYear: double (nullable = true)
 |-- types: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [ ]:
source_json_file_sparks_df.show(5)

+-------------------+--------------------+--------------------+-----------+-----+------+--------------------+----------+------------+
|               date|           localName|                name|countryCode|fixed|global|            counties|launchYear|       types|
+-------------------+--------------------+--------------------+-----------+-----+------+--------------------+----------+------------+
|2025-01-01 00:00:00|      New Year's Day|      New Year's Day|         US|false|  true|                    |      NULL|    [Public]|
|2025-01-20 00:00:00|Martin Luther Kin...|Martin Luther Kin...|         US|false|  true|                    |      NULL|    [Public]|
|2025-02-12 00:00:00|  Lincoln's Birthday|  Lincoln's Birthday|         US|false| false|US-CA,US-CT,US-IL...|      NULL|[Observance]|
|2025-02-17 00:00:00|Washington's Birt...|      Presidents Day|         US|false|  true|                    |      NULL|    [Public]|
|2025-04-18 00:00:00|         Good Friday|         Good Friday

In [ ]:
from pyspark.sql.functions import col, to_date
source_json_file_sparks_df = source_json_file_sparks_df.withColumn("date",to_date(col("date"),"yyyy-MM-dd"))

In [ ]:
source_json_file_sparks_df.show(5)

+----------+--------------------+--------------------+-----------+-----+------+--------------------+----------+------------+
|      date|           localName|                name|countryCode|fixed|global|            counties|launchYear|       types|
+----------+--------------------+--------------------+-----------+-----+------+--------------------+----------+------------+
|2025-01-01|      New Year's Day|      New Year's Day|         US|false|  true|                    |      NULL|    [Public]|
|2025-01-20|Martin Luther Kin...|Martin Luther Kin...|         US|false|  true|                    |      NULL|    [Public]|
|2025-02-12|  Lincoln's Birthday|  Lincoln's Birthday|         US|false| false|US-CA,US-CT,US-IL...|      NULL|[Observance]|
|2025-02-17|Washington's Birt...|      Presidents Day|         US|false|  true|                    |      NULL|    [Public]|
|2025-04-18|         Good Friday|         Good Friday|         US|false| false|US-CT,US-DE,US-HI...|      NULL|    [Public]|


In [ ]:
from pyspark.sql.functions import month,year,col,weekday
source_json_file_sparks_df = source_json_file_sparks_df.withColumn("holiday_month",month(col("date")))


In [ ]:
from pyspark.sql.functions import month,year,col,weekday
source_json_file_sparks_df = source_json_file_sparks_df.withColumn("weekday",date_format(col("date"),"EEEE"))

In [ ]:
display(source_json_file_sparks_df)

date,localName,name,countryCode,fixed,global,counties,launchYear,types,holiday_month,weekday
2025-01-01,New Year's Day,New Year's Day,US,false,true,,null,List(Public),1,Wednesday
2025-01-20,"Martin Luther King, Jr. Day","Martin Luther King, Jr. Day",US,false,true,,null,List(Public),1,Monday
2025-02-12,Lincoln's Birthday,Lincoln's Birthday,US,false,false,"US-CA,US-CT,US-IL,US-IN,US-KY,US-MI,US-NY,US-MO,US-OH",null,List(Observance),2,Wednesday
2025-02-17,Washington's Birthday,Presidents Day,US,false,true,,null,List(Public),2,Monday
2025-04-18,Good Friday,Good Friday,US,false,false,"US-CT,US-DE,US-HI,US-IN,US-KY,US-LA,US-NC,US-ND,US-NJ,US-TN",null,List(Public),4,Friday
2025-04-18,Good Friday,Good Friday,US,false,false,US-TX,null,List(Optional),4,Friday
2025-05-08,Truman Day,Truman Day,US,false,false,US-MO,null,"List(School, Authorities)",5,Thursday
2025-05-26,Memorial Day,Memorial Day,US,false,true,,null,List(Public),5,Monday
2025-06-19,Juneteenth National Independence Day,Juneteenth National Independence Day,US,false,true,,null,List(Public),6,Thursday
2025-07-04,Independence Day,Independence Day,US,false,true,,null,List(Public),7,Friday


In [ ]:
source_json_file_sparks_df.groupBy("weekday").agg(count("*").alias("count_of_holiday")).orderBy(col("count_of_holiday").desc()).show()

+---------+----------------+
|  weekday|count_of_holiday|
+---------+----------------+
|   Monday|               6|
| Thursday|               4|
|   Friday|               3|
|Wednesday|               2|
|  Tuesday|               1|
+---------+----------------+



In [ ]:
display(source_json_file_sparks_df)

date,localName,name,countryCode,fixed,global,counties,launchYear,types,holiday_month,weekday
2025-01-01,New Year's Day,New Year's Day,US,false,true,,null,List(Public),1,Wednesday
2025-01-20,"Martin Luther King, Jr. Day","Martin Luther King, Jr. Day",US,false,true,,null,List(Public),1,Monday
2025-02-12,Lincoln's Birthday,Lincoln's Birthday,US,false,false,"US-CA,US-CT,US-IL,US-IN,US-KY,US-MI,US-NY,US-MO,US-OH",null,List(Observance),2,Wednesday
2025-02-17,Washington's Birthday,Presidents Day,US,false,true,,null,List(Public),2,Monday
2025-04-18,Good Friday,Good Friday,US,false,false,"US-CT,US-DE,US-HI,US-IN,US-KY,US-LA,US-NC,US-ND,US-NJ,US-TN",null,List(Public),4,Friday
2025-04-18,Good Friday,Good Friday,US,false,false,US-TX,null,List(Optional),4,Friday
2025-05-08,Truman Day,Truman Day,US,false,false,US-MO,null,"List(School, Authorities)",5,Thursday
2025-05-26,Memorial Day,Memorial Day,US,false,true,,null,List(Public),5,Monday
2025-06-19,Juneteenth National Independence Day,Juneteenth National Independence Day,US,false,true,,null,List(Public),6,Thursday
2025-07-04,Independence Day,Independence Day,US,false,true,,null,List(Public),7,Friday


In [ ]:
month_holiday = source_json_file_sparks_df.groupBy("holiday_month").agg(count("*").alias("count_of_holidays")).filter(col("count_of_holidays")>2).orderBy("holiday_month")
display(month_holiday)


holiday_month,count_of_holidays


In [ ]:
source_json_file_sparks_df.write.mode("overwrite").json("abfss://assignment-container@myazuredemostorageacc.dfs.core.windows.net/bronze_assignment/public_holidays/json")
source_json_file_sparks_df.write.mode("overwrite").parquet("abfss://assignment-container@myazuredemostorageacc.dfs.core.windows.net/bronze_assignment/public_holidays/parquet")